In [5]:
!pip install transformers torch datasets
!pip install tqdm
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, pipeline, AutoTokenizer, AutoModel, AutoModelForQuestionAnswering, Trainer, TrainingArguments, AutoModelForQuestionAnswering, BertForQuestionAnswering
from sentence_transformers import SentenceTransformer
# from datasets import load_dataset, Dataset
import torch
import torch.nn.functional as F
import re
from nltk.tokenize import sent_tokenize
import nltk
import json
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

os.environ["WANDB_DISABLED"] = "true"
nltk.download('punkt')
nltk.download('punkt_tab')
from google.colab import drive
drive.mount('/content/drive')




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Mounted at /content/drive


Tokenize text at sentence level for BERT tokenizer.  This is generic and will run for any relevant path so do it first.


In [6]:
def load_text(file_path):
    print("Loading the text from the file...")
    with open(book_path, 'r', encoding='utf-8') as file:
        return file.read()

def remove_gutenberg_header_footer(text, start_marker, end_marker):
    start_index = text.find(start_marker)
    if start_index == -1:
        return text

    end_index = text.find(end_marker)
    if end_index == -1:
        return text

    cleaned_text = text[start_index + len(start_marker):end_index].strip()
    return cleaned_text

book_path = "/content/drive/My Drive/Cosc524 - Collaboration/a study in scarlet.txt"

# here we are cleaning the text and removing the headers and footers
raw_text = load_text(book_path)

print(raw_text[0:20])

start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK A STUDY IN SCARLET ***"

cleaned_text = remove_gutenberg_header_footer(raw_text, start_marker, end_marker)

print("Cleaned text sample:\n")

cleaned_text = cleaned_text[740:]
print(cleaned_text[0:725])

def tokenize_sentences(text):
    print("Tokenizing the text into sentences...")
    sentences = sent_tokenize(text)
    print(f"Total sentences: {len(sentences)}")
    return sentences

#Removing the table of contents so that RAG no longer tries to generate context with them.
my_sentences = tokenize_sentences(cleaned_text)
my_sentences = my_sentences[29:]

Loading the text from the file...
﻿The Project Gutenbe
Cleaned text sample:

_Being a reprint from the Reminiscences of_ JOHN H. WATSON, M.D.,
_Late of the Army Medical Department._)




CHAPTER I.
MR. SHERLOCK HOLMES.


In the year 1878 I took my degree of Doctor of Medicine of the
University of London, and proceeded to Netley to go through the course
prescribed for surgeons in the army. Having completed my studies there,
I was duly attached to the Fifth Northumberland Fusiliers as Assistant
Surgeon. The regiment was stationed in India at the time, and before I
could join it, the second Afghan war had broken out. On landing at
Bombay, I learned that my corps had advanced through the passes, and
was already deep in the enemy’s country. I followed, however, with many
other officers who were i
Tokenizing the text into sentences...
Total sentences: 2179


Recursive Chunking Implementation

In [7]:
from transformers import AutoTokenizer

# Initialize the tokenizer for 'all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Set the maximum token limit for each chunk (you can adjust it as needed)
MAX_TOKENS = 128

def recursive_chunk_with_overlap(text, max_tokens=128, overlap=50):
    """
    Chunk text with a specified overlap between chunks.

    Args:
        text (str): The input text to chunk.
        max_tokens (int): Maximum number of tokens allowed per chunk.
        overlap (int): Number of overlapping tokens between chunks.

    Returns:
        list[str]: A list of overlapping text chunks.
    """
    tokens = tokenizer.tokenize(text)

    if len(tokens) <= max_tokens:
        return [text]

    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap  # Slide window with overlap

    return chunks


# Example usage: Chunk the entire book

text_chunks = recursive_chunk_with_overlap(cleaned_text)

# Print the number of chunks and a sample chunk
print(f"Number of chunks: {len(text_chunks)}")
print(f"Sample chunk: {text_chunks[0]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (55708 > 512). Running this sequence through the model will result in indexing errors


Number of chunks: 715
Sample chunk: _ being a reprint from the reminiscences of _ john h. watson, m. d., _ late of the army medical department. _ ) chapter i. mr. sherlock holmes. in the year 1878 i took my degree of doctor of medicine of the university of london, and proceeded to netley to go through the course prescribed for surgeons in the army. having completed my studies there, i was duly attached to the fifth northumberland fusiliers as assistant surgeon. the regiment was stationed in india at the time, and before i could join it, the second afghan war had broken out. on landing at bombay, i learned that


Document Based Chunking

This chunks based on sentences or paragraphs

In [8]:
from transformers import AutoTokenizer
import re

# Initialize the tokenizer for 'all-MiniLM-L6-v2' model
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Set the maximum token limit for each chunk (adjust as needed)
MAX_TOKENS = 256

def chunk_document_by_paragraph(text, max_tokens=MAX_TOKENS):

    # Split the document into paragraphs
    paragraphs = text.split('\n')  # Assuming paragraphs are separated by newlines
    chunks = []
    current_chunk = []

    # Iterate through the paragraphs and chunk them
    for paragraph in paragraphs:
        # Tokenize the paragraph
        tokenized_paragraph = tokenizer.tokenize(paragraph)

        # If adding this paragraph exceeds the token limit, start a new chunk
        if len(tokenized_paragraph) + sum(len(tokenizer.tokenize(p)) for p in current_chunk) > max_tokens:
            if current_chunk:  # If there's a current chunk, add it to the chunks list
                chunks.append(' '.join(current_chunk))
            current_chunk = [paragraph]  # Start a new chunk with the current paragraph
        else:
            current_chunk.append(paragraph)

    # Add the final chunk if any paragraphs remain
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

def chunk_document_by_sentence(text, max_tokens=MAX_TOKENS):

    # Split the document into sentences
    sentences = re.split(r'(?<=\.)\s+', text)  # Split by periods followed by a space (sentence delimiter)
    chunks = []
    current_chunk = []

    # Iterate through the sentences and chunk them
    for sentence in sentences:
        # Tokenize the sentence
        tokenized_sentence = tokenizer.tokenize(sentence)

        # If adding this sentence exceeds the token limit, start a new chunk
        if len(tokenized_sentence) + sum(len(tokenizer.tokenize(s)) for s in current_chunk) > max_tokens:
            if current_chunk:  # If there's a current chunk, add it to the chunks list
                chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]  # Start a new chunk with the current sentence
        else:
            current_chunk.append(sentence)

    # Add the final chunk if any sentences remain
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Choose either paragraph-based or sentence-based chunking:
paragraph_chunks = chunk_document_by_paragraph(cleaned_text)
sentence_chunks = chunk_document_by_sentence(cleaned_text)

# Print the result
print(f"Paragraph-based chunks: {len(paragraph_chunks)} chunks")
print(f"Sample paragraph chunk: {paragraph_chunks[0]}")

print(f"\nSentence-based chunks: {len(sentence_chunks)} chunks")
print(f"Sample sentence chunk: {sentence_chunks[0]}")


Paragraph-based chunks: 225 chunks
Sample paragraph chunk: _Being a reprint from the Reminiscences of_ JOHN H. WATSON, M.D., _Late of the Army Medical Department._)     CHAPTER I. MR. SHERLOCK HOLMES.   In the year 1878 I took my degree of Doctor of Medicine of the University of London, and proceeded to Netley to go through the course prescribed for surgeons in the army. Having completed my studies there, I was duly attached to the Fifth Northumberland Fusiliers as Assistant Surgeon. The regiment was stationed in India at the time, and before I could join it, the second Afghan war had broken out. On landing at Bombay, I learned that my corps had advanced through the passes, and was already deep in the enemy’s country. I followed, however, with many other officers who were in the same situation as myself, and succeeded in reaching Candahar in safety, where I found my regiment, and at once entered upon my new duties.  The campaign brought honours and promotion to many, but for me it had 

Semantic Chunking

In [9]:
from sentence_transformers import SentenceTransformer

# Initialize the tokenizer and model for 'all-MiniLM-L6-v2'
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_sentence_embeddings(sentences, model=None):
    embeddings = model.encode(sentences, show_progress_bar=True)
    return embeddings  # Shape: [num_sentences, embedding_dim]


def semantic_chunking(max_tokens=256, similarity_threshold=0.75, sentence_embeddings=None, sentences=my_sentences):
    # Initialize variables for chunking
    current_chunk = [sentences[0]]
    current_chunk_embedding = sentence_embeddings[0].reshape(1, -1)
    chunks = []

    # Iterate through the sentences and group them based on semantic similarity
    for i in range(1, len(sentences)):
        current_embedding = sentence_embeddings[i].reshape(1, -1)
        similarity = cosine_similarity(current_chunk_embedding, current_embedding).mean()

        if similarity > similarity_threshold and len(tokenizer.tokenize(' '.join(current_chunk + [sentences[i]]))) <= max_tokens:
            # If similar and within token limit, add the sentence to the current chunk
            current_chunk.append(sentences[i])
            current_chunk_embedding = np.mean([current_chunk_embedding, current_embedding], axis=0)
        else:
            # If not similar or exceeds token limit, finalize current chunk and start a new one
            chunks.append(' '.join(current_chunk))
            current_chunk = [sentences[i]]
            current_chunk_embedding = current_embedding

    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

my_embeddings = generate_sentence_embeddings(my_sentences, model=model)
semantic_chunks = semantic_chunking(sentence_embeddings=my_embeddings, sentences=my_sentences)

# Print the semantic chunks
print(f"Number of chunks: {len(semantic_chunks)}")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/68 [00:00<?, ?it/s]

Number of chunks: 2148


Generate sentence embeddings with SBERT.
We use this model because it was designed to create rich embeddings preserving semantic similarity



In [10]:
from sentence_transformers import SentenceTransformer

def generate_sentence_embeddings(sentences, model=None):
    embeddings = model.encode(sentences, show_progress_bar=True)
    return embeddings  # Shape: [num_sentences, embedding_dim]


Load sentence embeddings or generate your own.

SBERT will compute the embeddings very fast. Currently taking only a minute or so without a GPU.

In [11]:
model = SentenceTransformer('all-MiniLM-L6-v2')  # Or another SBERT model

# my_embeddings = torch.load('/content/drive/My Drive/Cosc524 - Collaboration/embeddings/bert_embeddings.pt')
my_embeddings = generate_sentence_embeddings(semantic_chunks, model=model)

Batches:   0%|          | 0/68 [00:00<?, ?it/s]

This is the driver code of RAG context generation.
Once we compute our embeddings we use the cosine similarity to understand which sentences are close in the embedded space and retreive the top 'k' closest sentences.  We also need to embed our query for comparison.

In [12]:
def retrieve_relevant_info(query, top_k=5, embeddings=None, sentences=None, model=None):

    embeddings = torch.tensor(embeddings)
    # Compute the query embedding
    query_embedding = torch.tensor(model.encode(query)).unsqueeze(0)  # Shape: [1, embedding_dim]

    # Compute cosine similarities between query and sentence embeddings
    similarities = F.cosine_similarity(embeddings, query_embedding, dim=1)

    # Get top-k most similar sentences
    top_indices = torch.topk(similarities, top_k).indices

    # Retrieve the relevant sentences
    relevant_sentences = [sentences[idx] for idx in top_indices]
    return relevant_sentences


Generate RAG context

In [13]:
#We define our query here so it is consistent throughout retreival and answering.
query = "Who is the protagonist in A Study in Scarlet?"

In [14]:
# Didn't end up using this generation for anything...
# For text generation
# generator = pipeline('text-generation', model='bert-base-uncased')

#The model specified here needs to be the same one used to create embeddings
#This is because we need to embed the query as well.
model = SentenceTransformer('all-MiniLM-L6-v2')

#You need to be extremely careful that your sentences map properly to your embeddings!!! I made this mistake!
relevant_sentences = retrieve_relevant_info(query, top_k=10, embeddings=my_embeddings, sentences=semantic_chunks, model=model)
context = ' '.join(relevant_sentences)

for sentence in relevant_sentences:
  print(sentence)

I might not have gone but for you, and so have
missed the finest study I ever came across: a study in scarlet, eh?
“That’s the result of all our Study in Scarlet: to get them a
testimonial!”

“Never mind,” I answered, “I have all the facts in my journal, and the
public shall know them.
Even the professional
detectives, _blasé_ as they were in every detail of crime, appeared to
be keenly interested in the man’s story.
There’s the scarlet thread of
murder running through the colourless skein of life, and our duty is to
unravel it, and isolate it, and expose every inch of it.
It might be made a
text-book for detectives to teach them what to avoid.”

I felt rather indignant at having two characters whom I had admired
treated in this cavalier style.
The
murderer has written it with his or her own blood.
I don’t want to be
remembered as a common cut-throat.”

The Inspector and the two detectives had a hurried discussion as to the
advisability of allowing him to tell his story.
The reader may

Testing different text generation models

In [15]:
# !pip uninstall dataset
# !pip install fsspec==2024.10.0
# !pip install accelerate
# !pip install transformers
# !pip install autoawq
# !pip install transformers
# !pip install accelerate
# !pip install sentencepiece

In [16]:
# !huggingface-cli login

In [17]:
# !pip install autoawq
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# import autoawq

# model = "TheBloke/mistral-7b-instruct-v0.1-awq"
# tokenizer = AutoTokenizer.from_pretrained(model)
# model = AutoModelForCausalLM.from_pretrained(model)

In [18]:
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)


Testing someone elses fine tuned Bert for question answering.

In [19]:
# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).





Sherlock Holmes


Testing our fine tuned model

In [20]:
fine_tuned_model_path = "/content/drive/My Drive/Cosc524 - Collaboration/qa_model/finetuned"
dataset_path = "/content/drive/My Drive/Cosc524 - Collaboration/dataset/qa_dataset.json"

fine_tuned_model = AutoModelForQuestionAnswering.from_pretrained(fine_tuned_model_path)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

# Initialize tokenizer and QA pipeline
qa_pipeline = pipeline("question-answering", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)




.


Testing out of the box Bert

In [21]:
# Initialize tokenizer and QA pipeline
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)

# Get the answer
answer = qa_pipeline(question=query, context=context)["answer"]
print('\n\n')
print(answer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.





of it. It might
